In [3]:
import pandas as pd
import numpy as np
import altair as alt
from vega_datasets import data

In [4]:
data = pd.read_csv("day.csv")

In [5]:
data = data[["dteday", "season", "weekday", "weathersit", "temp", "hum", "windspeed", "casual", "registered"]]
data = pd.melt(data, id_vars = ["dteday", "season", "weekday", "weathersit", "temp", "hum", "windspeed"], value_vars = ["casual", "registered"], var_name = "user_type", value_name = "count")
data

,dteday,season,weekday,weathersit,temp,hum,windspeed,user_type,count
0,2011-01-01,1,6,2,0.344167,0.805833,0.160446,casual,331
1,2011-01-02,1,0,2,0.363478,0.696087,0.248539,casual,131
2,2011-01-03,1,1,1,0.196364,0.437273,0.248309,casual,120
3,2011-01-04,1,2,1,0.200000,0.590435,0.160296,casual,108
4,2011-01-05,1,3,1,0.226957,0.436957,0.186900,casual,82
...,...,...,...,...,...,...,...,...,...
1457,2012-12-27,1,4,2,0.254167,0.652917,0.350133,registered,1867
1458,2012-12-28,1,5,2,0.253333,0.590000,0.155471,registered,2451
1459,2012-12-29,1,6,2,0.253333,0.752917,0.124383,registered,1182
1460,2012-12-30,1,0,1,0.255833,0.483333,0.350754,registered,1432


In [6]:
data["temp_bin"] = pd.qcut(data["temp"], q = 5, labels = ["Very Cold", "Cold", "Temperate", "Warm", "Very Warm"])
data.head(100)

,dteday,season,weekday,weathersit,temp,hum,windspeed,user_type,count,temp_bin
0,2011-01-01,1,6,2,0.344167,0.805833,0.160446,casual,331,Cold
1,2011-01-02,1,0,2,0.363478,0.696087,0.248539,casual,131,Cold
2,2011-01-03,1,1,1,0.196364,0.437273,0.248309,casual,120,Very Cold
3,2011-01-04,1,2,1,0.200000,0.590435,0.160296,casual,108,Very Cold
4,2011-01-05,1,3,1,0.226957,0.436957,0.186900,casual,82,Very Cold
...,...,...,...,...,...,...,...,...,...,...
95,2011-04-06,2,3,1,0.390833,0.470833,0.263063,casual,413,Cold
96,2011-04-07,2,4,1,0.437500,0.602917,0.162312,casual,571,Temperate
97,2011-04-08,2,5,2,0.335833,0.836250,0.226992,casual,172,Cold
98,2011-04-09,2,6,2,0.342500,0.877500,0.133083,casual,879,Cold


In [7]:
["#2c7bb6", "#abd9e9", "#ffffbf", "#fdae61", "#d7191c"]

['#2c7bb6', '#abd9e9', '#ffffbf', '#fdae61', '#d7191c']

In [8]:
selection = alt.selection_multi(fields=["temp_bin"], bind = "legend")


base = alt.Chart(data).mark_area().encode(
    alt.X("dteday:T"),
    alt.Y("sum(count):Q"),
    alt.Color("temp_bin:O", scale = alt.Scale(range = ["turbo"])),
    opacity = alt.condition(selection, alt.value(1), alt.value(0.2)),
#     line = True
).properties(
    width = 1400,
    height = 350
).add_selection(
    selection
)



In [9]:
line = alt.Chart(data).mark_rule(color = "black").encode(
    y = "mean(sum(count):Q",
)

line

alt.Chart(...)

In [10]:
bar_users = alt.Chart(data).mark_bar().encode(
    x = "user_type:N",
    y = "average(count):Q",
    
).properties(
    width = 150,
    height = 550).transform_filter(
    selection
)

In [30]:
base & bar_users

alt.VConcatChart(...)

In [ ]:
points = alt.Chart(data).mark_circle().encode(
   alt.X(alt.repeat("column"), type = "quantitative"),
   alt.Y(alt.repeat("row"), type = "quantitative"),
   color='weekday:N',
).properties(
    width = 150,
    height = 150,
).repeat(
    row = ["temp", "cnt", "hum", "windspeed"],
    column = ["windspeed", "hum", "ag"]).interactive()

In [ ]:
points

In [ ]:
bars = alt.Chart(data).mark_bar().encode(
    y='weekday:N',
    x='cnt:Q',
    color = "weekday:O"
).interactive()

In [ ]:
bars

In [ ]:
brush = alt.selection_interval()

In [ ]:
scatter_temp = alt.Chart(data).mark_circle().encode(
   x = "temp:Q",
   y = "cnt:Q",
   color = alt.condition(brush, 'weekday:N', alt.value('lightgray')),
   tooltip = ["registered", "casual"]
).properties(
    width = 350,
    height = 350,
).add_selection(brush)

In [ ]:
scatter_temp

In [ ]:
bar_users = alt.Chart(data).mark_bar().encode(
    y= "weathersit:N"
).transform_filter(brush)

In [ ]:
scatter_temp & bar_users

In [ ]:
scatter_hum = alt.Chart(data).mark_circle().encode(
   x = "hum:Q",
   y = "cnt:Q",
   color = "weekday:N",
   tooltip = ["registered", "casual"]
).properties(
    width = 350,
    height = 350,
).interactive()

In [ ]:
scatter_hum

In [ ]:
scatter_wind = alt.Chart(data).mark_circle().encode(
   x = "windspeed:Q",
   y = "cnt:Q",
   color = "weekday:N",
   tooltip = ["registered", "casual"]
).properties(
    width = 350,
    height = 350,
).interactive()

In [ ]:
scatter_wind

In [ ]:
alt.Chart(data).mark_bar().encode(
    alt.X("occupation:N"),
    y='mean(wage):Q',
).configure_mark(
    opacity = 0.3,
    color = "green"
)